In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from tqdm import tqdm
from ryu_tool import mytool
from ryu_tool.min_variance import PortfolioOptimzer

In [ ]:
# 필요한 파일을 불러옵니다
stock_price = pd.read_pickle("Data/allstock_reduced.pickle")
spy_mask = pd.read_pickle("Data/spx_mask.pickle")

In [ ]:
portfolio = PortfolioOptimzer(price=stock_price,
                              spx_mask=spy_mask)

In [ ]:
pf_rtn, weight_df = portfolio.run_optimizer(start_year="2020",
                                            end_year="2023",
                                            rebalancing="Q",
                                            shrinkage_method="clipping",
                                            args={"k":5})

In [ ]:
mytool.draw_return_result({"title": pf_rtn})

In [ ]:
weight_df.iloc[:,:100].plot(figsize=(24,12))